## Collecting Anomaly Patterns for Evaludation of AD

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

In [5]:
import sys
sys.path.append('../')
from tsdr import tsdr
from eval import groundtruth

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
from meltria import loader

metrics_files = !find /datasets/argowf-chaos-rq54b/ -type f -name "*.json" 
dataset_generator = loader.load_dataset_as_generator(metrics_files, target_metric_types={
        "containers": True,
        "services": True,
        "nodes": True,
        "middlewares": True,
    },
    num_datapoints=120,
)

rerords = [r for rec in dataset_generator for r in rec]

In [ ]:
import pandas as pd
from collections import defaultdict

In [ ]:
route_by_case: dict[tuple[str, str], list[tuple[list[str], pd.DataFrame]]] = defaultdict(list)

for record in rerords:
    filtered_df: pd.DataFrame = tsdr.filter_out_no_change_metrics(record.data_df, parallel=True)
    gt_candidates = groundtruth.select_ground_truth_metrics_in_routes(record.pk, filtered_df.columns.to_list(), record.chaos_type(), record.chaos_comp())
    for routes, _ in gt_candidates:
        route_by_case[record.chaos_type(), record.chaos_comp()].append((routes, filtered_df))

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
SAVE_DIR = "../samples/anomaly_patterns"

In [ ]:
import datetime
import jsonlines
import glob

In [ ]:
def select_or_create_collection():
    jsonl_files = glob.glob(f"{SAVE_DIR}/*.jsonl")
    


In [ ]:
def create_widget(yield_on_click) -> widgets.Box:
    save_button = widgets.Button(description='Save')
    skip_button = widgets.Button(description='Skip')
    select_pattern = widgets.Select(
        options=[
            'Sudden increase', 'Sudden decrease', 'Level shift up', 'Level shift down', 
            'Steady increase', 'Steady decrease', 'Single spike', 'Single dip',
            'Transient level shift up', 'Transient level shift down', 'Multiple spikes', 'Multiple dips', 'Fluctuations',
            'White noise', 'Other normal',
        ],
        rows=15,
        description='Pattern:',
        layout=widgets.Layout(width='30%'),
    )
    select_position = widgets.Select(
        options=["no anomaly", "anomaly_during_the_chaos", "anomaly_in_before_chaos"],
        layout=widgets.Layout(width='30%'),
    )
    output = widgets.Output(layout={'border': '1px solid black'})

    fig = plt.figure(figsize=(6, 4), clear=True)
    ax = fig.add_subplot(1, 1, 1)

    def show() -> None:
        target_app, chaos_type, chaos_comp, metric, ts = next(yield_on_click)
        ax.clear()
        ax.set_title(f"{chaos_type}/{chaos_comp}\n{metric}\n")
        ax.plot(ts)
        with output:
            display(ax.get_figure())

    def on_save_click_callback(clicked_button: widgets.Button) -> None:
        yield_on_click.send(select_position.value, select_pattern.value)
        with output:
            output.clear_output(wait=True)
            print(f"Selected {select_pattern.value} and {select_position.value}!")
        show()

    save_button.on_click(on_save_click_callback)
    plt.close(fig=fig)
    show()

    def on_skip_click_callback(clicked_button: widgets.Button) -> None:
        with output:
            output.clear_output(wait=True)
            print(f"Skipped")
        show()

    skip_button.on_click(on_skip_click_callback)

    return widgets.HBox([widgets.HBox([select_position, select_pattern]), widgets.VBox([save_button, skip_button]), output])

In [ ]:
def gen_time_series(routes_by_case):
    now = datetime.datetime.today().strftime('%Y%m%d-%H%M%S')
    save_file_name = f"{SAVE_DIR}/anomaly_patterns_{now}.jsonl"
    # append mode
    writer = jsonlines.open(save_file_name, mode='a', flush=True)

    for (chaos_type, chaos_comp), routes in routes_by_case.items():
        unique_metric_check = {}
        for metrics, filtered_df in routes:
            for metric in metrics:
                # skip duplicated metric
                if metric in unique_metric_check and unique_metric_check[metric]:
                    continue
                unique_metric_check[metric] = True

                ts = filtered_df.loc[:, metric].to_numpy()
                position, pattern_name = (yield (record.target_app(), chaos_type, chaos_comp, metric, ts))
                if position is None or pattern_name is None:
                    continue
                writer.write({
                    'target_app': record.target_app(), 
                    'chaos_type': chaos_type,
                    'chaos_comp': chaos_comp, 
                    'metric': metric,
                    'anomaly_position': position,
                    'anomaly_pattern': pattern_name,
                    'time_series': ts.tolist(),
                })
    writer.close()

In [ ]:
box = create_widget(gen_time_series(route_by_case))
display(box)